# Root Finding and Optimization Methods

In computational physics and astrophysics, many problems reduce to two
fundamental kinds:
1. Root finding:
   Where does a function vanish?
   I.e., solve $f(x) = 0$.
2. Optimization:
   Where does a function reach an extremum (minimum or maximum)?
   I.e., solve $\nabla f(x) = 0$.

These two kind of problems are deeply connected.
Optimization often boils down to root finding on the derivative.
And root finding sometimes requires optimization-like strategies
to accelerate convergence.

Some classic examples of root finding include:
* Solving Kepler's equation $M = E - e \sin E$ to predict planetary
  orbits.
* Finding eigenfrequencies of stellar oscillations by locating roots
  of characteristic equations.

as well as optimization:
* Determining the launch angle of a projectile for maximum range.
* Fitting astrophysical models to observational data by minimizing a
  chi-square error function.
* Training machine learning models for data analysis in astronomy.

In simple cases, closed-form solutions exist (e.g. projectile motion
without air drag).
However, in realistic systems, equations are often nonlinear,
high-dimensional, and analytically unsolvable.
Numerical root finding and optimization methods are the only way to
solve these systems.

## General Framework of Interating Algorithms

Root finding means solving
\begin{align}
  f(x) = 0.
\end{align}
Many algorithms approach this through **iteration**:
starting from an initial guess, we repeatedly update $x$ until the
error is small.

### Fixed-Point Viewpoint

A powerful way to unify root-finding methods is to rewrite the problem
as a fixed-point equation:
\begin{align}
  x = g(x).
\end{align}

Then we can iterate:
\begin{align}
  x_{n+1} = g(x_n).
\end{align}

The solution $x^*$ is a *fixed point* of $g(x)$.
If the update rule is well chosen, the iteration converges to $x^*$.

### Convergence Criterion

Near the fixed point $x^*$, expand $g(x)$ in a Taylor series:
\begin{align}
  x_{n+1} = g(x_n) &\approx g(x^*) + g'(x^*) (x_n - x^*) = x^* + g'(x^*) (x_n - x^*).
\end{align}
Therefore,  
\begin{align}
  \frac{x_{n-1} - x^*}{x_n - x^*} &\approx g'(x^*).
\end{align}

It is clear that,
* If $|g'(x^*)| < 1$, the error shrinks, and the iteration converges.
* If $|g'(x^*)| > 1$, the iteration diverges.
* The closer $|g'(x^*)|$ is to 0, the faster the convergence.

This provides a general way to compare methods.

### Classical Root Finders

As we will soon see, classical root finding methods can be fitted into
this picture.
* Bisection Method:
  Is repeatedly shrinking an interval where the root must lie.
  The update rule is kind of a "double fixed-point scheme" where both
  the upper and lower bounds converge to the root.
  It is guaranteed to converge but only linearly.
* Newton–Raphson Method:
  Corresponds to choosing
  \begin{align}
    g(x) = x - \frac{f(x)}{f'(x)}.
  \end{align}
  If $f'(x^*) \neq 0$, this converges quadratically near the root.
* Secant Method:
  Uses the same Newton update rule, but replaces $f'(x)$ with a finite
  difference.
  This still fits into the fixed-point framework, with a convergence
  rate between bisection and Newton.

Thus, all root-finding methods can be viewed as different choices of
$g(x)$, with a trade-off between robustness and speed.

### Example

Let's solve $f(x)=x^2-2=0$.  
One possible choice of $g(x)$ is
\begin{align}
  g(x) = \tfrac12\left(x + \tfrac{2}{x}\right).
\end{align}
This is the
[ancient Babylonian update](https://www.sciencedirect.com/science/article/pii/S0315086022000477)
for $\sqrt{2}$.

In [ ]:
def g(x):
    return (x + 2/x)/2

x = 1.0
for i in range(5):
    print(f"Iteration {i}: x = {x}")
    x = g(x)

which converges very quickly to $\sqrt{2}$.

## Root Finding Methods

### Bisection Method

The Bisection Method is the simplest root-finding algorithm.
It trades speed for guaranteed convergence.
This makes it the "workhorse" method when robustness is more important
than efficiency.

Suppose $f(x)$ is continuous on an interval $[a,b]$.
If $f(a)$ and $f(b)$ have opposite signs, then by the
[Intermediate Value Theorem](https://en.wikipedia.org/wiki/Intermediate_value_theorem),
there exists at least one root in $(a,b)$.

The bisection method works by repeatedly halving the interval:
1. Compute the midpoint $m = (a+b)/2$.
2. Evaluate $f(m)$.
3. Select the half-interval $[a,m]$ or $[m,b]$ that contains the sign
   change.
4. Repeat until the interval is smaller than a desired tolerance.

Each step reduces the interval length by half:
\begin{align}
  (b-a) \to \tfrac12(b-a) \to \tfrac14(b-a) \to \cdots
\end{align}

After $n$ iterations, the uncertainty in the root is
\begin{align}
  \Delta x_n \approx \frac{b-a}{2^n}.
\end{align}

Although this convergence "exponentially" in terms of number of steps
$n$, we do not call this expoential convergence.
Instead, "convergence" in numerical analysis is usually from a step
size, i.e., $b-a$ for bisection method.
As $\Delta x_n$ scales only linear to $b-a$, bisection method is only
linear convergence.
It is reliable, but slower than other methods that we will introduce
later.

In [ ]:
def bisection(f, a, b, tol=1e-6, imax=100):
    
    if f(a)*f(b) >= 0:
        raise ValueError("f(a) and f(b) must have opposite signs.")
        
    for _ in range(imax):
        m = 0.5*(a+b)
        if f(m) == 0 or (b-a)/2 < tol:
            return m
        
        if f(a)*f(m) > 0:
            a = m
        else:
            b = m

    raise ValueError("Maximum iterations reached without convergence")

Let's solve $f(x) = x^3 − x − 2$,
which has a root between 1 and 2.

In [ ]:
def f(x):
    return x**3 - x - 2

root = bisection(f, 1, 2, tol=1e-6)
print("Approximate root:", root)
print("f(root) =", f(root))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

X = np.linspace(1, 2, 101)
Y = f(X)

plt.plot(X, Y, label="f(x)")
plt.plot(root, f(root), "o", label="Root")
plt.axhline(0, color="black", lw=1)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.legend()

The bisection method is the most robust way to refine a root by
repeatedly shrinking the search interval.
Because it is so basic, it is one of the algorithm explicitly required
by ASTR 513!

### Newton-Raphson Method

The Newton-Raphson Method is one of the most important and widely used
root-finding algorithms.

Unlike bisection, which only uses function values, Newton's method
leverages the derivative to achieve much faster convergence, but at a
cost of robustness.

Suppose we want to solve $f(x) = 0$.
Expand $f(x)$ around a current guess $x_n$ with a first-order Taylor
expansion:
\begin{align}
  f(x) \approx f(x_n) + f'(x_n)(x - x_n).
\end{align}

The root of this linear approximation occurs at:
\begin{align}
  x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}.
\end{align}

This is the Newton update rule.
It can also be seen as: "Draw the tangent line at $x_n$; where it
crosses the x-axis becomes $x_{n+1}$."

* Quadratic convergence:
  If the initial guess is close to the true root $x^*$, the error
  shrinks roughly like
  \begin{align}
    |x_{n+1}-x^*| \sim |x_n-x^*|^2,
  \end{align}
  meaning the number of correct digits roughly doubles at each step.

* Fragility:
  * If $f'(x_n)=0$, the method fails (division by zero).
  * If the initial guess is far from the root, the iteration may
    diverge or converge to the *wrong* root.

Thus, Newton's method is fast but fragile.

In [ ]:
def newton(f, fp, x0, tol=1e-6, imax=100, history=False):

    X = [x0]
    for _ in range(imax):
        fn, fpn = f(X[-1]), fp(X[-1])
        if fpn == 0:
            raise ValueError("Derivative is zero: Newton step undefined.")
            
        X.append(X[-1] - fn/fpn)
        if abs(X[-1] - X[-2]) < tol:
            return np.array(X) if history else X[-1]

    msg = "Maximum iterations reached without convergence"
    if history:
        from warnings import warn
        warn(msg)
        return np.array(X)
    else:
        raise ValueError(msg)

Let's solve $f(x) = x^3 − x − 2$ again so $f'(x) = 3x^2 - 1$.

In [ ]:
f  = lambda x:   x**3 - x - 2
fp = lambda x: 3*x**2 - 1

r  = newton(f, fp, x0=1)

print("Approximate root:", r)
print("f(root) =", f(r))

In [ ]:
def tangent(f, fp, x0):
    m = fp(x0)
    return lambda x: f(x0) + m*(x - x0)

X = np.linspace(0.9, 2.1, 221)
Y = f(X)
R = newton(f, fp, 1, history=True)

plt.axhline(0, color='k', ls=':', lw=1)
plt.plot(X, Y, color='k', label="f(x)")

for n in range(len(R)-1):
    plt.plot(R[n], f(R[n]), "o", label=f"Step {n}", color=f"C{n}")
    plt.plot(X, tangent(f, fp, R[n])(X),            color=f"C{n}")
    plt.plot([R[n+1], R[n+1]], [0, f(R[n+1])], ':', color=f"C{n}")
plt.plot(R[-1], f(R[-1]), "o", label=f"Step {len(R)-1}", color=f"C{len(R)-1}")

plt.xlabel("x")
plt.ylabel("f(x)")
plt.xlim(0.9, 2.1)
plt.ylim(-2.5, 4.5)
#plt.xlim( 1.5213, 1.5215)
#plt.ylim(-0.0005, 0.0005)
plt.legend()
plt.title("Newton–Raphson: tangent iteration")

Let's try different initial guesses:

In [ ]:
for x0 in np.linspace(-5, 5, 11):
    try:
        R = newton(f, fp, x0, history=True)
        print(f"Start {x0:.2f} -> root {R[-1]:.6f} in {len(R)} steps")
    except Exception as e:
        print(f"Failed: {e}; History: {R}")

Note that near the root 1.5, the convergence is very fast.
Starting at 0.0 took many more steps and almost fails because it
initially gives a poor direction.
The Newton-Raphson method may actually diverge.

In [ ]:
# HANDSON: try to provide a f() (and hence fp()) and x0 so that
#          Newton-Raphson fails to converge.
# HINT:    try f(x) = cos(x) - x


### Newton-Raphson Method with Automatic Differentiation (by JAX)

Computing derivatives manually is tedious.
With JAX, we can define only $f(x)$ and let autodiff handle $f'(x)$.

In [ ]:
import jax
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
from jax import grad

def autonewton(f, x0, tol=1e-6, imax=100, history=False):

    fp = grad(f)
    X  = [float(x0)]
    for _ in range(imax):
        fn, fpn = f(X[-1]), fp(X[-1])
        if fpn == 0:
            raise ValueError("Derivative is zero: Newton step undefined.")
            
        X.append(X[-1] - fn/fpn)
        if abs(X[-1] - X[-2]) < tol:
            return jnp.array(X) if history else X[-1]

    msg = "Maximum iterations reached without convergence"
    if history:
        from warnings import warn
        warn(msg)
        return jnp.array(X)
    else:
        raise ValueError(msg)

In [ ]:
r = autonewton(f, x0=1)
print("Approximate root:", r)
print("f(root) =", f(r))

**Pros:**
extremely fast (quadratic convergence) when near the root.

**Cons:**
requires derivative (not really a problem with autodiff), can fail
with bad initial guess.

**Best practice:**
combine with a robust method (e.g. start with bisection, then switch
to Newton).

### Newton-Raphson Method for Nonlinear Systems

So far, we have solved single equations $f(x)=0$.
But in real applications, from orbital mechanics to stellar structure
modeling, we often need to solve systems of nonlinear equations:
\begin{align}
  \mathbf{f}(\mathbf{x}) =
  \begin{bmatrix}
  f_1(x_1, x_2, \dots, x_n) \\
  f_2(x_1, x_2, \dots, x_n) \\
  \vdots \\
  f_n(x_1, x_2, \dots, x_n)
  \end{bmatrix} = \mathbf{0}.
\end{align}

In multiple dimensions, we generalize Newton's method by using the
Jacobian matrix:
\begin{align}
  J(\mathbf{x}) =
  \begin{bmatrix}
  \frac{\partial f_1}{\partial x_1} & \frac{\partial f_1}{\partial x_2} & \dots & \frac{\partial f_1}{\partial x_n} \\
  \frac{\partial f_2}{\partial x_1} & \frac{\partial f_2}{\partial x_2} & \dots & \frac{\partial f_2}{\partial x_n} \\
  \vdots & \vdots & \ddots & \vdots \\
  \frac{\partial f_n}{\partial x_1} & \frac{\partial f_n}{\partial x_2} & \dots & \frac{\partial f_n}{\partial x_n}
  \end{bmatrix}.
\end{align}

At each iteration, we solve the linear system:
\begin{align}
  J(\mathbf{x}_n)\,\Delta \mathbf{x} = -\mathbf{f}(\mathbf{x}_n),
\end{align}
and update:
\begin{align}
  \mathbf{x}_{n+1} = \mathbf{x}_n + \Delta \mathbf{x}.
\end{align}

This is the Newton–Raphson update for nonlinear systems.

In [ ]:
def newton_system(F, J, X0, tol=1e-6, imax=100, history=False):

    X = [np.array(X0, dtype=float)]
    for _ in range(imax):
        Fn = F(X[-1])
        Jn = J(X[-1])
        dX = np.linalg.solve(Jn, -Fn) # let numpy raise exception

        X.append(X[-1] + dX)
        if np.max(abs(X[-1] - X[-2])) < tol:
            return np.array(X) if history else X[-1]

    msg = "Maximum iterations reached without convergence"
    if history:
        from warnings import warn
        warn(msg)
        return np.array(X)
    else:
        raise ValueError(msg)

Consider the system:
\begin{align}
  f_1(x,y) &= x^2 + y^2 - 4 = 0 \\
  f_2(x,y) &= e^x + y − 1 = 0
\end{align}

In [ ]:
def F(X):
    x, y = X
    return np.array([
        x**2 + y**2 - 4,
        np.exp(x) + y - 1,
    ])

def J(X):
    x, y = X
    return np.array([
        [2*x,       2*y],
        [np.exp(x), 1.0],
    ])

X0   = [1.0, 1.0]
root = newton_system(F, J, X0)

print("Approximate root:", root)
print("F(root) =", F(root))

### Newton-Raphson Systems with Automatic Jacobian (by JAX)

In higher dimensions, computing derivatives by hand is even more
tedious.
We can use JAX autodiff to generate the Jacobian automatically.

In [ ]:
from jax import jacfwd

In [ ]:
def autonewton_system(F, X0, tol=1e-6, imax=100, history=False):

    J = jacfwd(F)
    X = [jnp.array(X0, dtype=float)]
    for _ in range(imax):
        Fn = F(X[-1])
        Jn = J(X[-1])
        dX = np.linalg.solve(Jn, -Fn) # let numpy raise exception

        X.append(X[-1] + dX)
        if np.max(abs(X[-1] - X[-2])) < tol:
            return npj.array(X) if history else X[-1]

    msg = "Maximum iterations reached without convergence"
    if history:
        from warnings import warn
        warn(msg)
        return npj.array(X)
    else:
        raise ValueError(msg)

In [ ]:
def F(X):
    x, y = X
    return jnp.array([
        x**2 + y**2 - 4,
        jnp.exp(x) + y - 1,
    ])

X0 = [1.0, 1.0]
R  = autonewton_system(F, X0)

print("Approximate root:", R)
print("F(root) =", F(R))

**Pros:**
* Quadratic convergence in multiple dimensions.
* Works naturally with systems of equations.

**Cons:**
* Requires solving a linear system at each step (costly for large
  $n$).
* Fragile if the Jacobian is singular or if the initial guess is poor.

In [ ]:
# HANDSON: Try modifying the system of equations to:
#          
#              f1(x,y) = sin(x) + y**2 - 1
#              f2(x,y) = x**2 - y - 1
#
#          Use both the hand-coded Jacobian and JAX autodiff.
#          How many iterations are needed to converge from an initial
#          guess [0.5, 0.5]?


## Optimization Methods

We now turn from root finding to optimization.

At first, these may seem like different problems:
* Root finding:
  Solve $f(x) = 0$.
* Optimization:
  Find $x^*$ that minimizes (or maximizes) $f(x)$.

But they are deeply connected.
A critical point of a differentiable function occurs when the gradient
vanishes:
\begin{align}
  \nabla f(x^*) = 0.
\end{align}
Thus, optimization can often be reformulated as root finding on the
gradient.

Some applications in astrophysics include:
* The principle of least action states that nature chooses
  trajectories that extremize the action.
* Fitting a model to data often requires minimizing a chi-square error
  function.
* Training a neural network to classify galaxies is an optimization
  problem, i.e., minimizing a loss function over millions of
  parameters.

### Gradient Descent in One Dimension

The most basic optimization algorithm is **Gradient Descent**.
It is simple, intuitive, and forms the foundation of modern
optimization in high dimensions.

Suppose we want to minimize a differentiable function $f(x)$.
* The derivative $f'(x)$ points in the direction of steepest ascent.
* Moving in the opposite direction reduces $f(x)$.

The update rule is:
\begin{align}
  x_{n+1} = x_n - \alpha f'(x_n),
\end{align}
where $\alpha > 0$ is the "learning rate" or "step size".

In [ ]:
def gd(fp, x0, alpha, tol=1e-6, imax=100, history=False):

    X = [x0]
    for _ in range(imax):
        X.append(X[-1] - alpha * fp(X[-1]))            
        if abs(X[-1] - X[-2]) < tol:
            return np.array(X) if history else X[-1]

    msg = "Maximum iterations reached without convergence"
    if history:
        from warnings import warn
        warn(msg)
        return np.array(X)
    else:
        raise ValueError(msg)

Let's consider:
\begin{align}
  f(x) = (x-3)^2 + 4,
\end{align}
which has a unique minimum at $x=3$.
The derivative is:
\begin{align}
  f'(x) = 2(x-3).
\end{align}

In [ ]:
f  = lambda x:   (x-3)**2 + 4
fp = lambda x: 2*(x-3)

In [ ]:
x0    = 0.0
alpha = 0.1
Xmin  = gd(fp, x0, alpha, history=True)

print("Approximate minimum:", Xmin[-1])
print("f(xmin) =", f(Xmin[-1]))

In [ ]:
X = np.linspace(-0.5, 3.5, 401)
Y = f(X)

plt.plot(X, Y, label="f(x)")
plt.plot(Xmin, f(Xmin), "o-", label="GD steps")
plt.xlabel("x")
plt.ylabel("f(x)")
plt.legend()

In [ ]:
# HANDSON: Change `x0` and `alpha` and monitor how many steps are
#          needed to obtain the solution.
#          What is the optimal choice of `alpha`?
# NOTE:    You may need to adjust the axis limits.


From the hands-on, we saw:
* If $\alpha$ is too small, convergence is very slow.
* If $\alpha$ is too large, the algorithm may overshoot and even
  diverge.
* With a well-chosen $\alpha$, the iteration converges smoothly to the
  minimum.

This trade-off between stability and speed is central to all
gradient-based optimization.

Compared gradient descent with Newton-Raphson method, there are few
interesting observations:

* Newton-Raphson requires function evaluation $f(x)$ but not learning
  rate $\alpha$.
  Graident descent is the opposite.

* Both of them use fix point iterations:
  \begin{align}
    x_{n+1} = x_n -
    \begin{cases}
      f(x_n) / f'(x_n) & \text{(Newton-Raphson)} \\
      \alpha f'(x_n)   & \text{(Gradient descent)}
    \end{cases}
  \end{align}

* If we consider gradient descent as root finding for $g(x) = f'(x)$,
  then Newton-Raphson becomes
  \begin{align}
    x_{n+1} &= x_n - g(x_n) / g'(x_n) \\
            &= x_n - f'(x_n) / f''(x_n)
  \end{align}
  Comparing this to gradient descent, we have $\alpha = 1/f''(x_n)$,
  which makes sense because $f''(x_n) > 0$ only when $x_n$ is near its
  minimum.

* If we consider $h(x) = f(x)^2$ so $h'(x) = 2 f(x) f'(x)$, then
  gradient descent becomes
  \begin{align}
    x_{n+1} &= x_n - \alpha h'(x_n) \\
            &= x_n - 2 \alpha f(x_n) f'(x_n)
  \end{align}
  Comparing this to Newton-Raphson, we have $\alpha = 1/2[f'(x_n)]^2$.

In [ ]:
# HANDSON: solve a root finding problem using gradient descent.
#          Does it work better or worse?


In [ ]:
# HANDSON: solve a minimization problem using root finding.
#          Does it work better or worse?
